<a href="https://colab.research.google.com/github/jessiechd/RAG_Model/blob/main/0303_supabase_similaritysearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install "vecs[text_embedding]" --q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [3]:
import vecs
from vecs.adapter import Adapter, ParagraphChunker, TextEmbedding

# create vector store client
DB_CONNECTION = ""

vx = vecs.create_client(DB_CONNECTION)

# create a collection with an adapter
docs1 = vx.get_or_create_collection(
    name="docs1",
    adapter=Adapter(
        [
            ParagraphChunker(skip_during_query=True),
            TextEmbedding(model='all-MiniLM-L6-v2'),
        ]
    )
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
import psycopg2



In [78]:
import json

def upsert_json_to_vector_store(json_file, docs):
    conn = psycopg2.connect(DB_CONNECTION)
    cursor = conn.cursor()
    # Load JSON data
    with open(json_file, "r") as f:
        data = json.load(f)  # Load JSON file into a list of dictionaries

    # Add metadata with the source filename
    records = []
    for item in data:
      chunk_id = f"{item['metadata']['source']}_{item['chunk_id']}"

      if 'content' in item:
        embedding_vector = item['content']
        if isinstance(embedding_vector, dict):  # Check if it's a dictionary
            embedding_vector = str(embedding_vector)  # Convert to string
      elif 'table' in item:
        embedding_vector = item['table']
        if isinstance(embedding_vector, dict):  # Check if it's a dictionary
            embedding_vector = str(embedding_vector)

      metadata = {
          "source": item['metadata']['source'],
          "section": item['metadata']['section']
      }

      content = item.get('content', None)
      tabel = item.get('table', None)

      # Convert to JSON string for PostgreSQL insertion
      metadata_json = json.dumps(metadata)
      tabel_json = json.dumps(tabel) if tabel else None

      # Insert content into `doc1` table
      cursor.execute("""
          INSERT INTO public.doc1 (chunk_id, content, tabel, metadata)
          VALUES (%s, %s, %s, %s)
          RETURNING id
      """, (chunk_id, content, tabel_json, metadata_json))

      document_id = cursor.fetchone()[0]

      conn.commit()

      print(f"Document inserted with ID: {document_id}")


      records.append((chunk_id, embedding_vector, metadata))

    # Insert into vector store
    docs.upsert(records=records)
    print(f"Inserted {len(records)} records successfully from {json_file}.")
    cursor.close()
    conn.close()


In [79]:
# Example usage
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (EN)/17.json", docs1)


Document inserted with ID: 1
Document inserted with ID: 2
Document inserted with ID: 3
Document inserted with ID: 4
Document inserted with ID: 5
Document inserted with ID: 6
Document inserted with ID: 7
Document inserted with ID: 8
Document inserted with ID: 9
Document inserted with ID: 10
Document inserted with ID: 11
Document inserted with ID: 12
Document inserted with ID: 13
Document inserted with ID: 14
Inserted 14 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (EN)/17.json.


# EN: 12/12

In [80]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (EN)/2014-monarch-plus-service-manual.json", docs1)

Document inserted with ID: 15
Document inserted with ID: 16
Document inserted with ID: 17
Document inserted with ID: 18
Document inserted with ID: 19
Document inserted with ID: 20
Document inserted with ID: 21
Document inserted with ID: 22
Document inserted with ID: 23
Document inserted with ID: 24
Document inserted with ID: 25
Document inserted with ID: 26
Document inserted with ID: 27
Document inserted with ID: 28
Document inserted with ID: 29
Document inserted with ID: 30
Document inserted with ID: 31
Document inserted with ID: 32
Document inserted with ID: 33
Document inserted with ID: 34
Document inserted with ID: 35
Document inserted with ID: 36
Document inserted with ID: 37
Document inserted with ID: 38
Inserted 24 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (EN)/2014-monarch-plus-service-manual.json.


In [81]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (EN)/2024_11_05_Ferrari_Q3_2024_Results_Press_Release.json", docs1)

Document inserted with ID: 39
Document inserted with ID: 40
Document inserted with ID: 41
Document inserted with ID: 42
Document inserted with ID: 43
Document inserted with ID: 44
Document inserted with ID: 45
Document inserted with ID: 46
Document inserted with ID: 47
Document inserted with ID: 48
Document inserted with ID: 49
Inserted 11 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (EN)/2024_11_05_Ferrari_Q3_2024_Results_Press_Release.json.


In [82]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (EN)/231161_OperationsMaintenanceManual.json", docs1)

Document inserted with ID: 50
Document inserted with ID: 51
Document inserted with ID: 52
Inserted 3 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (EN)/231161_OperationsMaintenanceManual.json.


In [83]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (EN)/PDF1.json", docs1)

Document inserted with ID: 53
Document inserted with ID: 54
Document inserted with ID: 55
Document inserted with ID: 56
Document inserted with ID: 57
Document inserted with ID: 58
Document inserted with ID: 59
Document inserted with ID: 60
Document inserted with ID: 61
Document inserted with ID: 62
Document inserted with ID: 63
Document inserted with ID: 64
Document inserted with ID: 65
Document inserted with ID: 66
Document inserted with ID: 67
Document inserted with ID: 68
Document inserted with ID: 69
Document inserted with ID: 70
Document inserted with ID: 71
Document inserted with ID: 72
Document inserted with ID: 73
Document inserted with ID: 74
Document inserted with ID: 75
Inserted 23 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (EN)/PDF1.json.


In [84]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (EN)/README.json", docs1)

Document inserted with ID: 76
Inserted 1 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (EN)/README.json.


In [85]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (EN)/SUPO-744_REV_A.json", docs1)

Document inserted with ID: 77
Document inserted with ID: 78
Document inserted with ID: 79
Document inserted with ID: 80
Document inserted with ID: 81
Document inserted with ID: 82
Document inserted with ID: 83
Document inserted with ID: 84
Document inserted with ID: 85
Document inserted with ID: 86
Document inserted with ID: 87
Document inserted with ID: 88
Document inserted with ID: 89
Document inserted with ID: 90
Document inserted with ID: 91
Document inserted with ID: 92
Document inserted with ID: 93
Document inserted with ID: 94
Document inserted with ID: 95
Document inserted with ID: 96
Document inserted with ID: 97
Document inserted with ID: 98
Document inserted with ID: 99
Document inserted with ID: 100
Document inserted with ID: 101
Document inserted with ID: 102
Document inserted with ID: 103
Document inserted with ID: 104
Document inserted with ID: 105
Document inserted with ID: 106
Document inserted with ID: 107
Document inserted with ID: 108
Document inserted with ID: 109


In [86]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (EN)/Test-OCR-Handwritten.json", docs1)

Document inserted with ID: 112
Inserted 1 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (EN)/Test-OCR-Handwritten.json.


In [87]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (EN)/VVS005s_030s_AHU_EN.json", docs1)

Document inserted with ID: 113
Document inserted with ID: 114
Document inserted with ID: 115
Document inserted with ID: 116
Document inserted with ID: 117
Document inserted with ID: 118
Document inserted with ID: 119
Document inserted with ID: 120
Document inserted with ID: 121
Document inserted with ID: 122
Document inserted with ID: 123
Document inserted with ID: 124
Document inserted with ID: 125
Document inserted with ID: 126
Document inserted with ID: 127
Document inserted with ID: 128
Document inserted with ID: 129
Document inserted with ID: 130
Document inserted with ID: 131
Document inserted with ID: 132
Document inserted with ID: 133
Document inserted with ID: 134
Document inserted with ID: 135
Document inserted with ID: 136
Document inserted with ID: 137
Document inserted with ID: 138
Document inserted with ID: 139
Document inserted with ID: 140
Document inserted with ID: 141
Document inserted with ID: 142
Document inserted with ID: 143
Document inserted with ID: 144
Document

In [88]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (EN)/ai-in-america-oai-economic-blueprint-20250113.json", docs1)

Document inserted with ID: 181
Document inserted with ID: 182
Document inserted with ID: 183
Document inserted with ID: 184
Document inserted with ID: 185
Document inserted with ID: 186
Document inserted with ID: 187
Document inserted with ID: 188
Document inserted with ID: 189
Document inserted with ID: 190
Document inserted with ID: 191
Document inserted with ID: 192
Inserted 12 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (EN)/ai-in-america-oai-economic-blueprint-20250113.json.


In [89]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (EN)/creatingsystem.json", docs1)

Document inserted with ID: 193
Document inserted with ID: 194
Inserted 2 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (EN)/creatingsystem.json.


In [90]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (EN)/image1.json", docs1)

Document inserted with ID: 195
Inserted 1 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (EN)/image1.json.


# SP: 13/13

In [91]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (SP)/04) CDR - Bilancio 31.12.17-with-image-refs.json", docs1)

Document inserted with ID: 196
Document inserted with ID: 197
Document inserted with ID: 198
Document inserted with ID: 199
Document inserted with ID: 200
Document inserted with ID: 201
Document inserted with ID: 202
Document inserted with ID: 203
Document inserted with ID: 204
Document inserted with ID: 205
Document inserted with ID: 206
Document inserted with ID: 207
Document inserted with ID: 208
Document inserted with ID: 209
Document inserted with ID: 210
Document inserted with ID: 211
Document inserted with ID: 212
Document inserted with ID: 213
Document inserted with ID: 214
Document inserted with ID: 215
Document inserted with ID: 216
Document inserted with ID: 217
Document inserted with ID: 218
Document inserted with ID: 219
Document inserted with ID: 220
Document inserted with ID: 221
Document inserted with ID: 222
Document inserted with ID: 223
Document inserted with ID: 224
Document inserted with ID: 225
Document inserted with ID: 226
Document inserted with ID: 227
Document

In [92]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (SP)/2015_5458_manutenzione_immobili-with-image-refs.json", docs1)

Document inserted with ID: 364
Document inserted with ID: 365
Document inserted with ID: 366
Inserted 3 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (SP)/2015_5458_manutenzione_immobili-with-image-refs.json.


In [93]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (SP)/Bilancio di previsione 2016 Assemblea 22-12-15-with-image-refs.json", docs1)

Document inserted with ID: 367
Document inserted with ID: 368
Document inserted with ID: 369
Document inserted with ID: 370
Document inserted with ID: 371
Document inserted with ID: 372
Document inserted with ID: 373
Document inserted with ID: 374
Document inserted with ID: 375
Document inserted with ID: 376
Document inserted with ID: 377
Document inserted with ID: 378
Inserted 12 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (SP)/Bilancio di previsione 2016 Assemblea 22-12-15-with-image-refs.json.


In [94]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (SP)/Brochure_PagoinRetePa-with-image-refs.json", docs1)

Document inserted with ID: 379
Document inserted with ID: 380
Document inserted with ID: 381
Document inserted with ID: 382
Document inserted with ID: 383
Document inserted with ID: 384
Inserted 6 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (SP)/Brochure_PagoinRetePa-with-image-refs.json.


In [95]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (SP)/Comunicato-Stampa-approvazione-2023-with-image-refs.json", docs1)

Document inserted with ID: 385
Document inserted with ID: 386
Document inserted with ID: 387
Document inserted with ID: 388
Document inserted with ID: 389
Document inserted with ID: 390
Document inserted with ID: 391
Document inserted with ID: 392
Document inserted with ID: 393
Document inserted with ID: 394
Document inserted with ID: 395
Inserted 11 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (SP)/Comunicato-Stampa-approvazione-2023-with-image-refs.json.


In [96]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (SP)/ESEMPIO 740-with-image-refs.json", docs1)

Document inserted with ID: 396
Document inserted with ID: 397
Document inserted with ID: 398
Document inserted with ID: 399
Document inserted with ID: 400
Document inserted with ID: 401
Document inserted with ID: 402
Document inserted with ID: 403
Document inserted with ID: 404
Document inserted with ID: 405
Document inserted with ID: 406
Inserted 11 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (SP)/ESEMPIO 740-with-image-refs.json.


In [97]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (SP)/MODELLO REDDITI 2020 PER 2019 RAGAZZOLI-with-image-refs.json", docs1)

Document inserted with ID: 407
Document inserted with ID: 408
Document inserted with ID: 409
Document inserted with ID: 410
Document inserted with ID: 411
Document inserted with ID: 412
Document inserted with ID: 413
Document inserted with ID: 414
Document inserted with ID: 415
Document inserted with ID: 416
Document inserted with ID: 417
Document inserted with ID: 418
Document inserted with ID: 419
Document inserted with ID: 420
Document inserted with ID: 421
Document inserted with ID: 422
Document inserted with ID: 423
Document inserted with ID: 424
Document inserted with ID: 425
Document inserted with ID: 426
Document inserted with ID: 427
Document inserted with ID: 428
Document inserted with ID: 429
Document inserted with ID: 430
Document inserted with ID: 431
Document inserted with ID: 432
Document inserted with ID: 433
Document inserted with ID: 434
Document inserted with ID: 435
Document inserted with ID: 436
Document inserted with ID: 437
Document inserted with ID: 438
Document

In [98]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (SP)/Manuale ROTOMARR-with-image-refs.json", docs1)

Document inserted with ID: 451
Document inserted with ID: 452
Document inserted with ID: 453
Document inserted with ID: 454
Document inserted with ID: 455
Document inserted with ID: 456
Document inserted with ID: 457
Inserted 7 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (SP)/Manuale ROTOMARR-with-image-refs.json.


In [99]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (SP)/Manuale-IRIS_SLIM_IN_TEC_IT-with-image-refs.json", docs1)

Document inserted with ID: 458
Document inserted with ID: 459
Document inserted with ID: 460
Document inserted with ID: 461
Document inserted with ID: 462
Document inserted with ID: 463
Document inserted with ID: 464
Document inserted with ID: 465
Document inserted with ID: 466
Document inserted with ID: 467
Document inserted with ID: 468
Document inserted with ID: 469
Document inserted with ID: 470
Document inserted with ID: 471
Document inserted with ID: 472
Document inserted with ID: 473
Document inserted with ID: 474
Document inserted with ID: 475
Document inserted with ID: 476
Document inserted with ID: 477
Document inserted with ID: 478
Document inserted with ID: 479
Document inserted with ID: 480
Document inserted with ID: 481
Document inserted with ID: 482
Document inserted with ID: 483
Document inserted with ID: 484
Document inserted with ID: 485
Document inserted with ID: 486
Document inserted with ID: 487
Document inserted with ID: 488
Inserted 31 records successfully from /

In [100]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (SP)/UNICO 2014 REDDITI 2013 FALCOMATA_ GIUSEPPE-with-image-refs.json", docs1)

Document inserted with ID: 489
Document inserted with ID: 490
Document inserted with ID: 491
Document inserted with ID: 492
Document inserted with ID: 493
Document inserted with ID: 494
Document inserted with ID: 495
Document inserted with ID: 496
Document inserted with ID: 497
Document inserted with ID: 498
Document inserted with ID: 499
Document inserted with ID: 500
Document inserted with ID: 501
Document inserted with ID: 502
Document inserted with ID: 503
Document inserted with ID: 504
Document inserted with ID: 505
Document inserted with ID: 506
Inserted 18 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (SP)/UNICO 2014 REDDITI 2013 FALCOMATA_ GIUSEPPE-with-image-refs.json.


In [101]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (SP)/contratto artuso-with-image-refs.json", docs1)

Document inserted with ID: 507
Document inserted with ID: 508
Document inserted with ID: 509
Document inserted with ID: 510
Document inserted with ID: 511
Document inserted with ID: 512
Document inserted with ID: 513
Inserted 7 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (SP)/contratto artuso-with-image-refs.json.


In [102]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (SP)/esempio-modello-fattura-with-image-refs.json", docs1)

Document inserted with ID: 514
Document inserted with ID: 515
Inserted 2 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (SP)/esempio-modello-fattura-with-image-refs.json.


In [103]:
upsert_json_to_vector_store("/content/drive/MyDrive/RAG - source files/md & arti (SP)/modello-730-2020-FERIGO-FRANCESCA-with-image-refs.json", docs1)

Document inserted with ID: 516
Document inserted with ID: 517
Document inserted with ID: 518
Document inserted with ID: 519
Document inserted with ID: 520
Document inserted with ID: 521
Document inserted with ID: 522
Document inserted with ID: 523
Document inserted with ID: 524
Document inserted with ID: 525
Document inserted with ID: 526
Document inserted with ID: 527
Document inserted with ID: 528
Document inserted with ID: 529
Document inserted with ID: 530
Document inserted with ID: 531
Document inserted with ID: 532
Document inserted with ID: 533
Document inserted with ID: 534
Document inserted with ID: 535
Document inserted with ID: 536
Inserted 21 records successfully from /content/drive/MyDrive/RAG - source files/md & arti (SP)/modello-730-2020-FERIGO-FRANCESCA-with-image-refs.json.


# Query embeddings

In [8]:
# search by text
docs1.query(data="what is the reason behind medical training in Africa")

['17.md_13_para_000',
 '17.md_3_para_000',
 '17.md_14_para_000',
 '17.md_2_para_000',
 '17.md_12_para_000',
 '17.md_6_para_000',
 '17.md_7_para_000',
 '17.md_8_para_000',
 '17.md_10_para_000',
 '17.md_1_para_000']

In [10]:
# search by text
docs1.query(data="pros and cons of machine learning")

['ai-in-america-oai-economic-blueprint-20250113.md_10_para_000',
 'ai-in-america-oai-economic-blueprint-20250113.md_6_para_000',
 'PDF1.md_5_para_000',
 'PDF1.md_13_para_000',
 'ai-in-america-oai-economic-blueprint-20250113.md_12_para_000',
 'ai-in-america-oai-economic-blueprint-20250113.md_7_para_000',
 'ai-in-america-oai-economic-blueprint-20250113.md_9_para_000',
 'ai-in-america-oai-economic-blueprint-20250113.md_4_para_000',
 'PDF1.md_22_para_000',
 'ai-in-america-oai-economic-blueprint-20250113.md_8_para_000']

# Retrieve original text

In [16]:
!pip install transformers torch

In [17]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load the MiniLM model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def generate_query_embedding(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

    # Generate embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the embeddings from the last hidden state
    embeddings = outputs.last_hidden_state

    # Average the embeddings to get a single vector
    query_embedding = torch.mean(embeddings, dim=1).squeeze().numpy()

    return query_embedding

# Example usage
input_string = "machine learning uses"
query_embedding = generate_query_embedding(input_string)

print("Query Embedding:", query_embedding)

Query Embedding: [-3.68123353e-01 -2.12523490e-02  3.01195383e-01  3.34497169e-02
  3.34910929e-01  2.50493348e-01  2.24344581e-01 -2.03185052e-01
 -1.21848896e-01  4.02191207e-02 -5.51119745e-01  1.59157276e-01
  1.16171576e-01 -2.01670215e-01 -2.86633253e-01 -5.63105941e-03
 -9.12084132e-02  3.80603403e-01 -2.94339985e-01 -7.11566567e-01
 -1.90295368e-01 -3.00559044e-01 -4.80764955e-01  3.18920285e-01
  1.31854370e-01 -1.81919858e-01 -1.80775020e-02  1.97066471e-01
  2.16266349e-01 -8.15517157e-02  1.30725786e-01 -7.39886910e-02
  2.44793028e-01  3.06188166e-01 -9.64777946e-01 -1.45785809e-02
 -5.14110923e-03  2.14979842e-01 -6.73895702e-02 -1.01595499e-01
 -5.50201833e-01 -6.62945092e-01  1.01455823e-01  9.05158445e-02
  1.06825089e+00  4.46812809e-01 -2.23689005e-01 -3.57650459e-01
 -2.56769899e-02  3.78968537e-01 -7.21421242e-01 -2.25668758e-01
  2.37117454e-01 -9.65114832e-02 -1.80485010e-01  1.89522445e-01
  3.55562031e-01  3.16216354e-03  2.67964989e-01 -1.23289205e-01
  1.0675

In [58]:
import json
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(DB_CONNECTION)
cursor = conn.cursor()


# Perform a vector search in the vecs schema

def vector_search1(query_embedding):  # Euclidean distance
    # Convert NumPy array to a list before serializing to JSON
    query_embedding_list = query_embedding.tolist()

    cursor.execute("""
        SELECT id FROM vecs.docs1
        ORDER BY (VEC <-> %s) LIMIT 5
    """, (json.dumps(query_embedding_list),))  # Pass the list to json.dumps()

    results = cursor.fetchall()
    return [row[0] for row in results]  # Return list of chunk_ids


def vector_search2(query_embedding):  # negative inner product
    # Convert NumPy array to a list before serializing to JSON
    query_embedding_list = query_embedding.tolist()

    cursor.execute("""
        SELECT id FROM vecs.docs1
        ORDER BY (VEC <#> %s) LIMIT 5
    """, (json.dumps(query_embedding_list),))  # Pass the list to json.dumps()

    results = cursor.fetchall()
    return [row[0] for row in results]  # Return list of chunk_ids


def vector_search3(query_embedding):  # 	cosine distance
    # Convert NumPy array to a list before serializing to JSON
    query_embedding_list = query_embedding.tolist()

    cursor.execute("""
        SELECT id FROM vecs.docs1
        ORDER BY (VEC <=> %s) LIMIT 5
    """, (json.dumps(query_embedding_list),))  # Pass the list to json.dumps()

    results = cursor.fetchall()
    return [row[0] for row in results]  # Return list of chunk_ids


# Fetch content from the public.doc1 table using chunk_id
def fetch_content(chunk_ids):
    if not chunk_ids:
        return []

    # Create a string of chunk_ids for the SQL IN clause

    chunk_ids = [chunk_id.replace("_para_000", "") for chunk_id in chunk_ids]
    chunk_ids_str = ', '.join(f"'{chunk_id}'" for chunk_id in chunk_ids)

    cursor.execute(f"""
    SELECT COALESCE(content, tabel::text) AS content
    FROM public.doc1
    WHERE chunk_id IN ({chunk_ids_str})
    """)


    return cursor.fetchall()

# Example usage  # Replace with your actual query embedding
# Example usage
input_string = "medical training"
query_embedding = generate_query_embedding(input_string)


In [59]:
chunk_ids = vector_search1(query_embedding)
content_results = fetch_content(chunk_ids)


# # Output the results
for content in content_results:
    print("Content:", content[0], "\n")  # Assuming content is the first column

Content: ## Introduction  
This article reports on the findings of a workshop held at the joint 5th World Organisation of Family Doctors (WONCA) Africa and 20th National Family Practitioners Conference in Tshwane, South Africa, in August 2017. Postgraduate training for family medicine in Africa takes place in the clinical workplace at the bedside or next to the patient in the clinic, district hospital or regional hospital. 1,2 Direct observation of the registrar by the supervisor with a learning conversation that evokes reflection and provides feedback is central to both training and assessment processes. 3  
Worldwide,  workplace-based  assessment  (WBA)  and  learning  has  been  recognised  as  the preferred way to train and assess postgraduate students. 4 It is well known that learning and assessment are not dependent on any one method, but require a variety of educational strategies to deliver on the curriculum, which are best embedded in clinical practice as an authentic context.

In [60]:
chunk_ids = vector_search2(query_embedding)
content_results = fetch_content(chunk_ids)


# # Output the results
for content in content_results:
    print("Content:", content[0], "\n")  # Assuming content is the first column

Content: ## Introduction  
This article reports on the findings of a workshop held at the joint 5th World Organisation of Family Doctors (WONCA) Africa and 20th National Family Practitioners Conference in Tshwane, South Africa, in August 2017. Postgraduate training for family medicine in Africa takes place in the clinical workplace at the bedside or next to the patient in the clinic, district hospital or regional hospital. 1,2 Direct observation of the registrar by the supervisor with a learning conversation that evokes reflection and provides feedback is central to both training and assessment processes. 3  
Worldwide,  workplace-based  assessment  (WBA)  and  learning  has  been  recognised  as  the preferred way to train and assess postgraduate students. 4 It is well known that learning and assessment are not dependent on any one method, but require a variety of educational strategies to deliver on the curriculum, which are best embedded in clinical practice as an authentic context.

In [61]:
chunk_ids = vector_search3(query_embedding)
content_results = fetch_content(chunk_ids)


# # Output the results
for content in content_results:
    print("Content:", content[0], "\n")  # Assuming content is the first column

Content: ## Introduction  
This article reports on the findings of a workshop held at the joint 5th World Organisation of Family Doctors (WONCA) Africa and 20th National Family Practitioners Conference in Tshwane, South Africa, in August 2017. Postgraduate training for family medicine in Africa takes place in the clinical workplace at the bedside or next to the patient in the clinic, district hospital or regional hospital. 1,2 Direct observation of the registrar by the supervisor with a learning conversation that evokes reflection and provides feedback is central to both training and assessment processes. 3  
Worldwide,  workplace-based  assessment  (WBA)  and  learning  has  been  recognised  as  the preferred way to train and assess postgraduate students. 4 It is well known that learning and assessment are not dependent on any one method, but require a variety of educational strategies to deliver on the curriculum, which are best embedded in clinical practice as an authentic context.

In [ ]:
# Commit the transaction and close the connection
conn.commit()
cursor.close()
conn.close()

https://supabase.github.io/vecs/concepts_adapters/

https://supabase.com/blog/vecs

https://supabase.com/docs/guides/ai/semantic-search

